# **Track the Beat on the Fly**

In [22]:
import numpy as np
from madmom.features.beats import DBNBeatTrackingProcessor, RNNBeatProcessor
from madmom.models import BEATS_LSTM
from madmom.processors import IOProcessor, process_online
import time 
from codelab_adapter_client import AdapterNode


class MyNode(AdapterNode):
    NODE_ID = "eim/beats"

    def __init__(self):
        super().__init__()
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)

node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)

node.send_data("start")

time.sleep(2)

kwargs = dict(
    fps = 100,
    correct = True,
    infile = None,
    outfile = None,
    #max_bpm = 170,
    #min_bpm = 100,
    nn_files = [BEATS_LSTM[0],BEATS_LSTM[1], BEATS_LSTM[2]],
    num_frames = 1,
    online = True,
    #verbose = 1
)

 
def beat_callback(beats, output=None):
    if len(beats) > 0:
        #global n
        # Do something with the beat (for now, just print the array to stdout)
        #n = n + 1
        #print(beats)
        node.send_data("beats") 
    
    
in_processor = RNNBeatProcessor(**kwargs)
beat_processor = DBNBeatTrackingProcessor(**kwargs)
out_processor = [beat_processor, beat_callback]
processor = IOProcessor(in_processor, out_processor)
process_online(processor, **kwargs)
    


In [ ]:
from madmom.features.beats import DBNBeatTrackingProcessor, RNNBeatProcessor 
from madmom.features.downbeats import DBNDownBeatTrackingProcessor, RNNDownBeatProcessor
from madmom.models import BEATS_LSTM, DOWNBEATS_BLSTM
from madmom.processors import IOProcessor, process_online
import time 
from codelab_adapter_client import AdapterNode


class MyNode(AdapterNode):
    NODE_ID = "eim/beats"

    def __init__(self):
        super().__init__()
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)

node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)

kwargs = dict(
    online = True,
    correct = True,
    fps = 100,
    infile = None,
    outfile = None,
    #beats_per_bar=[3,4]
   
)


def beat_callback(beats,output=None):
    if len(beats) > 0:
        # Do something with the beat (for now, just print the array to stdout)
        print(beats)
        #beats=["beats!"]
        #node.send_data(beats)
        

rnn = RNNBeatProcessor(**kwargs)
dbn = DBNBeatTrackingProcessor(**kwargs)
out_processor = [dbn, beat_callback]
in_processor = rnn
processor = IOProcessor(in_processor, out_processor)
process_online(processor, **kwargs)